In [ ]:
from jupyter_dash import JupyterDash
import dash
import dash_leaflet
import plotly.express as px
import pandas as pd
print("All imports OK")

In [4]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Dash and related imports
import dash_leaflet as dl
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output, State
import plotly.express as px
import base64

# Other libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import your CRUD class
from crud_module import CRUD

###########################
# Data Manipulation / Model
###########################
username = "aacuser"
password = "Sputnik1957"
shelter = CRUD(username, password)
host = "localhost"  
port = "27017"  
database = "aac" 
# Connect to database via CRUD Module
db = CRUD(username, password)

# Retrieve all documents initially
df = pd.DataFrame.from_records(db.read_query({}))

# Drop '_id' if present
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('GraziosoSalvareDashboard')

# Encode the Grazioso Salvare logo
image_filename = "Grazioso Salvare Logo.png"  # Ensure this file is in the same folder
encoded_image = base64.b64encode(open(image_filename, 'rb').read()).decode()

app.layout = html.Div([
    # Logo
    html.Img(src='data:image/png;base64,{}'.format(encoded_image), style={'height': '100px'}),
    # Title
    html.Center(html.B(html.H1('CS-340 Dashboard - [Aspen]'))),
    html.Hr(),

    # Filtering options (Radio buttons, etc.)
    html.Div([
    html.Label("Select Rescue Type:"),
    dcc.RadioItems(
        id='filter-type',
        options=[
            {'label': 'Reset', 'value': 'RESET'},
            {'label': 'Water Rescue', 'value': 'WATER'},
            {'label': 'Mountain or Wilderness Rescue', 'value': 'MOUNTAIN'},
            {'label': 'Disaster or Individual Tracking', 'value': 'DISASTER'},
        ],
        value='RESET',
        labelStyle={'display': 'inline-block', 'margin-right': '20px'}
    )
]),

    html.Hr(),

    # Data Table
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        page_size=10,
        style_table={'overflowX': 'auto'},
        row_selectable="single",
        selected_rows=[0],
        sort_action="native",
        filter_action="native"
    ),
    html.Br(),
    html.Hr(),

    # Row with chart and map side-by-side
    html.Div(
        className='row',
        style={'display': 'flex'},
        children=[
            html.Div(id='graph-id', className='col s12 m6'),
            html.Div(id='map-id', className='col s12 m6')
        ]
    )
])

#############################################
# Interaction Between Components / Controller
#############################################

# 1) Filter the data table based on rescue type
@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):
    """
    Simple filtering logic based on the selected rescue type.
    Replace with your actual queries as needed.
    """
    if filter_type == 'RESET':
        query = {}
    elif filter_type == 'WATER':
        query = {"breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever"]}}
    elif filter_type == 'MOUNTAIN':
        query = {"breed": {"$in": ["German Shepherd", "Alaskan Malamute"]}}
    elif filter_type == 'DISASTER':
        query = {"breed": {"$in": ["Doberman Pinscher", "Golden Retriever", "Border Collie"]}}
    else:
        query = {}

    records = db.read_query(query)

    dff = pd.DataFrame.from_records(records)

    if "_id" in dff.columns:
        dff.drop(columns=["_id"], inplace=True)

    return dff.to_dict('records')


# 2) Create a pie chart from the current table data
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):
    if not viewData:
        return "No Data Available"
    # Group by breed, count how many rows per breed
    dff = pd.DataFrame.from_dict(viewData)
    dff_count = dff.groupby("breed")["breed"].count().reset_index(name="count")
    # Sort descending and keep the top 10
    dff_count = dff_count.sort_values("count", ascending=False).head(10)

    # Create the pie chart using the aggregated data
    fig = px.pie(
        dff_count,
        names="breed",
        values="count",
        title="Top 10 Breeds"
    )

    return [dcc.Graph(figure=fig)]
# 3) Highlight selected columns in the data table
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if not selected_columns:  # if None or empty
        return []
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# 4) Update the geo-location chart for the selected row
@app.callback(
    Output('map-id', "children"),
    [
        Input('datatable-id', "derived_virtual_data"),
        Input('datatable-id', "derived_virtual_selected_rows")
    ]
)
def update_map(viewData, selected_rows):
    if not viewData:
        return "No Data Available"

    dff = pd.DataFrame.from_dict(viewData)

    # If no row is selected, default to row = 0
    if not selected_rows:
        row = 0
    else:
        row = selected_rows[0]

    # Check for enough columns to reference lat/long
    if len(dff.columns) < 15:
        return "Not enough columns for lat/long."

    # Center on Henderson, NV [36.04, -114.98]
    return [
        dl.Map(
            style={'width': '1000px', 'height': '500px'},
            center=[36.04, -114.98],
            zoom=10,
            children=[
                dl.TileLayer(id="base-layer-id"),
                dl.Marker(
                    position=[dff.iloc[row, 13], dff.iloc[row, 14]],
                    
                    children=[
                        dl.Tooltip(str(dff.iloc[row, 4])),
                        dl.Popup([
                            html.H1("Animal Name"),
                            html.P(str(dff.iloc[row, 9]))
                        ])
                    ]
                )
            ]
        )
    ]


if __name__ == '__main__':
    app.run_server(debug=True, port=8050)


ServerSelectionTimeoutError: nv-desktop-services.apporto.com:33016: [Errno 11001] getaddrinfo failed (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30s, Topology Description: <TopologyDescription id: 6963168483c3eb62f4ec8662, topology_type: Unknown, servers: [<ServerDescription ('nv-desktop-services.apporto.com', 33016) server_type: Unknown, rtt: None, error=AutoReconnect('nv-desktop-services.apporto.com:33016: [Errno 11001] getaddrinfo failed (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms)')>]>

In [5]:
import sys
print(sys.executable)

import pymongo, pandas
print("pymongo:", pymongo.__version__)
print("pandas:", pandas.__version__)



C:\Users\aspen\AppData\Local\Programs\Python\Python310\python.exe
pymongo: 4.16.0
pandas: 2.3.3


In [6]:
from pymongo import MongoClient

client = MongoClient("mongodb://localhost:27017/")
print(client.admin.command("ping"))


{'ok': 1.0}


In [7]:
db = client["aac"]
print("Collections:", db.list_collection_names())
print("Animal count:", db.animals.count_documents({}))

Collections: []
Animal count: 0
